In [ ]:
%pip install keras plotly


In [ ]:
%pip install keras --upgrade


In [ ]:
from io import StringIO
from numpy import array, arange, linspace, mean, std  
from pandas import DataFrame, read_csv  
from keras.models import Sequential  
from keras.layers import Dense 
from plotly.express import scatter, bar, line  
from plotly.subplots import make_subplots
from plotly.graph_objects import Figure, Scatter, Bar  
from seaborn import heatmap, pairplot  
import pandas as pd


In [ ]:
chicago_taxi_dataset = pd.read_csv("https://download.mlcc.google.com/mledu-datasets/chicago_taxi_train.csv")

In [ ]:
training_df = chicago_taxi_dataset[['TRIP_MILES', 'TRIP_SECONDS', 'FARE', 'COMPANY', 'PAYMENT_TYPE', 'TIP_RATE']]

print('Read dataset completed successfully.')
print('Total number of rows: {0}\n\n'.format(len(training_df.index)))
training_df.head(200)